In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import requests
import defs

In [10]:
fx_pairs = ['AUD_JPY', 'NZD_JPY', 'EUR_JPY', 'GBP_JPY', 'USD_JPY', 'AUD_USD', 'AUD_CAD', 'NZD_USD', 'EUR_USD', 'GBP_USD', 'USD_CAD']
timeframes = ['H1', 'H4', 'D']
session = requests.Session()
fx_dataframes = {}
trend_data_list = []
trend_df_columns = ['Asset'] + [f'MOMO_{timeframe}' for timeframe in timeframes]

for instrument in fx_pairs:
    trend_data = {'Asset': instrument}
    for timeframe in timeframes:
        url = f'{defs.OANDA_URL}/instruments/{instrument}/candles'
        params = {'count': 60, 'granularity': timeframe, 'price': "M"}
        response = session.get(url, params=params, headers=defs.SECURE_HEADER)

        if response.status_code == 200:
            data = response.json()
            my_data = [{'time': candle['time'], 'open': float(candle['mid']['o']), 'high': float(candle['mid']['h']), 'low': float(candle['mid']['l']), 'close': float(candle['mid']['c']), 'volume': candle['volume']} for candle in data['candles'] if candle['complete']]
            precision = 3 if 'JPY' in instrument else 5
            df = pd.DataFrame(my_data)
            df['time'] = pd.to_datetime(df['time'])
            df.set_index('time', inplace=True)
            df['SMA_FAST'] = ta.sma(df['close'], length=20).round(precision)
            df['SMA_SLOW'] = ta.sma(df['close'], length=50).round(precision)
            df['MOMO'] = np.where(df['SMA_FAST'] > df['SMA_SLOW'], 'Uptrend', 'Downtrend')
            fx_dataframes[(instrument, timeframe)] = df
            trend_data[f'MOMO_{timeframe}'] = df['MOMO'].iloc[-1]
        else:
            print(f"Failed to fetch data for {instrument} with granularity {timeframe}: {response.status_code}")
            trend_data[f'MOMO_{timeframe}'] = 'Error'

    trend_data_list.append(trend_data)

trend_df = pd.DataFrame(trend_data_list, columns=trend_df_columns)
trend_df.set_index('Asset', inplace=True)

print(trend_df)


           MOMO_H1    MOMO_H4     MOMO_D
Asset                                   
AUD_JPY    Uptrend    Uptrend    Uptrend
NZD_JPY    Uptrend  Downtrend    Uptrend
EUR_JPY    Uptrend    Uptrend    Uptrend
GBP_JPY    Uptrend    Uptrend    Uptrend
USD_JPY    Uptrend    Uptrend    Uptrend
AUD_USD  Downtrend  Downtrend  Downtrend
AUD_CAD  Downtrend  Downtrend    Uptrend
NZD_USD  Downtrend  Downtrend    Uptrend
EUR_USD  Downtrend  Downtrend    Uptrend
GBP_USD  Downtrend  Downtrend    Uptrend
USD_CAD  Downtrend    Uptrend    Uptrend


In [12]:
fx_dataframes['AUD_CAD', 'H1'].head()

,open,high,low,close,volume,SMA_FAST,SMA_SLOW,MOMO
time,,,,,,,,
2024-03-14 12:00:00+00:00,0.89171,0.89237,0.89054,0.89137,4915,NaN,NaN,Downtrend
2024-03-14 13:00:00+00:00,0.89138,0.89142,0.89001,0.89062,4811,NaN,NaN,Downtrend
2024-03-14 14:00:00+00:00,0.89057,0.89078,0.88940,0.88964,3633,NaN,NaN,Downtrend
2024-03-14 15:00:00+00:00,0.88963,0.89027,0.88904,0.88997,3718,NaN,NaN,Downtrend
2024-03-14 16:00:00+00:00,0.88997,0.89080,0.88987,0.89044,2738,NaN,NaN,Downtrend
